<a href="https://colab.research.google.com/github/rshamsini/COVID-19detection/blob/main/Classification_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Mounting the Google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np #mathematical operations over arrays
import pandas as pd #a powerful data analysis and manipulation library for Python
import os
import glob #glob module is used to retrieve files/pathnames matching a specified pattern.
from pathlib import Path

In [ ]:
#Reading train_image_level csv file
data = pd.read_csv('/content/drive/MyDrive/Optum/train_image_level.csv')

In [ ]:
data.head()

,id,boxes,label,StudyInstanceUID
0,000a312787f2_image,"[{'x': 789.28836, 'y': 582.43035, 'width': 102...",opacity 1 789.28836 582.43035 1815.94498 2499....,5776db0cec75
1,000c3a3f293f_image,NaN,none 1 0 0 1 1,ff0879eb20ed
2,0012ff7358bc_image,"[{'x': 677.42216, 'y': 197.97662, 'width': 867...",opacity 1 677.42216 197.97662 1545.21983 1197....,9d514ce429a7
3,001398f4ff4f_image,"[{'x': 2729, 'y': 2181.33331, 'width': 948.000...",opacity 1 2729 2181.33331 3677.00012 2785.33331,28dddc8559b2
4,001bd15d1891_image,"[{'x': 623.23328, 'y': 1050, 'width': 714, 'he...",opacity 1 623.23328 1050 1337.23328 2156 opaci...,dfd9fdd85a3e


In [ ]:
# Removing "_image" from id column
for i in range(len(data)):
    t=data.loc[i,"id"]
    data.loc[i,"id"]=t[:-6]

In [ ]:
data.head()

,id,boxes,label,StudyInstanceUID
0,000a312787f2,"[{'x': 789.28836, 'y': 582.43035, 'width': 102...",opacity 1 789.28836 582.43035 1815.94498 2499....,5776db0cec75
1,000c3a3f293f,NaN,none 1 0 0 1 1,ff0879eb20ed
2,0012ff7358bc,"[{'x': 677.42216, 'y': 197.97662, 'width': 867...",opacity 1 677.42216 197.97662 1545.21983 1197....,9d514ce429a7
3,001398f4ff4f,"[{'x': 2729, 'y': 2181.33331, 'width': 948.000...",opacity 1 2729 2181.33331 3677.00012 2785.33331,28dddc8559b2
4,001bd15d1891,"[{'x': 623.23328, 'y': 1050, 'width': 714, 'he...",opacity 1 623.23328 1050 1337.23328 2156 opaci...,dfd9fdd85a3e


In [ ]:
#Reading train_study_level csv file
data2 = pd.read_csv('/content/drive/MyDrive/Optum/train_study_level.csv')

In [ ]:
data2.head()

,id,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance
0,00086460a852_study,0,1,0,0
1,000c9c05fd14_study,0,0,0,1
2,00292f8c37bd_study,1,0,0,0
3,005057b3f880_study,1,0,0,0
4,0051d9b12e72_study,0,0,0,1


In [ ]:
# Removing "_study" from id column
for i in range(len(data2)):
    t=data2.loc[i,"id"]
    data2.loc[i,"id"]=t[:-6]

In [ ]:
data2.head()

,id,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance
0,00086460a852,0,1,0,0
1,000c9c05fd14,0,0,0,1
2,00292f8c37bd,1,0,0,0
3,005057b3f880,1,0,0,0
4,0051d9b12e72,0,0,0,1


In [ ]:
#Merging Both csv files by matching id and StudyInstanceID of both csv's respectively

df = pd.merge(left=data, right=data2, left_on='StudyInstanceUID', right_on='id')
df.head()

,id_x,boxes,label,StudyInstanceUID,id_y,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance
0,000a312787f2,"[{'x': 789.28836, 'y': 582.43035, 'width': 102...",opacity 1 789.28836 582.43035 1815.94498 2499....,5776db0cec75,5776db0cec75,0,1,0,0
1,000c3a3f293f,NaN,none 1 0 0 1 1,ff0879eb20ed,ff0879eb20ed,1,0,0,0
2,0012ff7358bc,"[{'x': 677.42216, 'y': 197.97662, 'width': 867...",opacity 1 677.42216 197.97662 1545.21983 1197....,9d514ce429a7,9d514ce429a7,0,1,0,0
3,001398f4ff4f,"[{'x': 2729, 'y': 2181.33331, 'width': 948.000...",opacity 1 2729 2181.33331 3677.00012 2785.33331,28dddc8559b2,28dddc8559b2,0,0,0,1
4,001bd15d1891,"[{'x': 623.23328, 'y': 1050, 'width': 714, 'he...",opacity 1 623.23328 1050 1337.23328 2156 opaci...,dfd9fdd85a3e,dfd9fdd85a3e,0,1,0,0


In [ ]:
#class 0-->Negative for Pneumonia 
#class 1-->Typical Appearance 
#class 2-->Indeterminate Appearance 
#class 3-->Atypical Appearance

res=[]

for i in range(len(df)) :
  if(df.loc[i,"Negative for Pneumonia"]):
    res.append(0)
  elif(df.loc[i,"Typical Appearance"]):
    res.append(1)
  elif(df.loc[i,"Indeterminate Appearance"]):
    res.append(2)
  elif(df.loc[i,"Atypical Appearance"]):
    res.append(3)

df['result']=res

In [ ]:
#Drop unnecessary Columns
df=df.drop(['Negative for Pneumonia','Typical Appearance','Indeterminate Appearance','Atypical Appearance'],axis=1)

In [ ]:
df.head()

,id_x,boxes,label,StudyInstanceUID,id_y,result
0,000a312787f2,"[{'x': 789.28836, 'y': 582.43035, 'width': 102...",opacity 1 789.28836 582.43035 1815.94498 2499....,5776db0cec75,5776db0cec75,1
1,000c3a3f293f,NaN,none 1 0 0 1 1,ff0879eb20ed,ff0879eb20ed,0
2,0012ff7358bc,"[{'x': 677.42216, 'y': 197.97662, 'width': 867...",opacity 1 677.42216 197.97662 1545.21983 1197....,9d514ce429a7,9d514ce429a7,1
3,001398f4ff4f,"[{'x': 2729, 'y': 2181.33331, 'width': 948.000...",opacity 1 2729 2181.33331 3677.00012 2785.33331,28dddc8559b2,28dddc8559b2,3
4,001bd15d1891,"[{'x': 623.23328, 'y': 1050, 'width': 714, 'he...",opacity 1 623.23328 1050 1337.23328 2156 opaci...,dfd9fdd85a3e,dfd9fdd85a3e,1


In [ ]:
# Copy of df(data)
data=df

In [ ]:
len(data)

6334

In [ ]:
path = '/content/drive/MyDrive/Optum/images/'
 
d=Path(path)
# This would print all the files and directories
images=[]
for files in d.glob('*/*.png'):
  images.append(files)

In [ ]:
images[0:5]

[PosixPath('/content/drive/MyDrive/Optum/images/train/8881c07203fc.png'),
 PosixPath('/content/drive/MyDrive/Optum/images/train/08932dca1447.png'),
 PosixPath('/content/drive/MyDrive/Optum/images/train/8866f708d338.png'),
 PosixPath('/content/drive/MyDrive/Optum/images/train/8951eac18958.png'),
 PosixPath('/content/drive/MyDrive/Optum/images/train/8925b615fc62.png')]

In [ ]:
#Creating a DataFrame with Location and id of images

total=pd.DataFrame()
total['Location']=images
total['id']=total.apply(lambda row:str(row.Location)[-16:-4],axis=1)
total.head()

,Location,id
0,/content/drive/MyDrive/Optum/images/train/8881...,8881c07203fc
1,/content/drive/MyDrive/Optum/images/train/0893...,08932dca1447
2,/content/drive/MyDrive/Optum/images/train/8866...,8866f708d338
3,/content/drive/MyDrive/Optum/images/train/8951...,8951eac18958
4,/content/drive/MyDrive/Optum/images/train/8925...,8925b615fc62


In [ ]:
len(total)

6334

In [ ]:
# Merging data and total DataFrames by equating id's
complete=pd.merge(left=total, right=data, left_on='id', right_on='id_x')
complete.head(10)

,Location,id,id_x,boxes,label,StudyInstanceUID,id_y,result
0,/content/drive/MyDrive/Optum/images/train/8881...,8881c07203fc,8881c07203fc,NaN,none 1 0 0 1 1,b4dcf0375a27,b4dcf0375a27,0
1,/content/drive/MyDrive/Optum/images/train/0893...,08932dca1447,08932dca1447,NaN,none 1 0 0 1 1,f109811743c5,f109811743c5,0
2,/content/drive/MyDrive/Optum/images/train/8866...,8866f708d338,8866f708d338,"[{'x': 1559.4013, 'y': 50.88, 'width': 1280.48...",opacity 1 1559.4013 50.88 2839.88143 1772.3200...,6bc01f9a44a5,6bc01f9a44a5,1
3,/content/drive/MyDrive/Optum/images/train/8951...,8951eac18958,8951eac18958,"[{'x': 583.55566, 'y': 1586.66667, 'width': 14...",opacity 1 583.55566 1586.66667 2002.22233 3165...,a0927e51897c,a0927e51897c,1
4,/content/drive/MyDrive/Optum/images/train/8925...,8925b615fc62,8925b615fc62,"[{'x': 1592.45982, 'y': 1223.98355, 'width': 4...",opacity 1 1592.45982 1223.98355 2029.73948 178...,cbf3b338fae0,cbf3b338fae0,2
5,/content/drive/MyDrive/Optum/images/train/8867...,8867b5e6d09e,8867b5e6d09e,"[{'x': 546.71653, 'y': 1111.15789, 'width': 52...",opacity 1 546.71653 1111.15789 1069.45335 2086...,ed9fc7d10eba,ed9fc7d10eba,1
6,/content/drive/MyDrive/Optum/images/train/9027...,9027e541f1f3,9027e541f1f3,"[{'x': 400.05097, 'y': 365.08267, 'width': 468...",opacity 1 400.05097 365.08267 868.30915 1423.2...,023a30330136,023a30330136,1
7,/content/drive/MyDrive/Optum/images/train/9268...,9268cee8f850,9268cee8f850,"[{'x': 294.99073, 'y': 421.76107, 'width': 592...",opacity 1 294.99073 421.76107 887.89289 1335.9...,cd6d6e960d3d,cd6d6e960d3d,1
8,/content/drive/MyDrive/Optum/images/train/9319...,9319acd2ff28,9319acd2ff28,"[{'x': 320.13323, 'y': 486.27589, 'width': 587...",opacity 1 320.13323 486.27589 907.314500000000...,b13d71d1edde,b13d71d1edde,2
9,/content/drive/MyDrive/Optum/images/train/9304...,9304a960bfcf,9304a960bfcf,NaN,none 1 0 0 1 1,1e7e77b791bd,1e7e77b791bd,0


In [ ]:
len(complete)

6334

In [ ]:
#Dropping repeated columns
complete.drop(columns=['id_x','id_y'],inplace=True)

In [ ]:
complete.head()

,Location,id,boxes,label,StudyInstanceUID,result
0,/content/drive/MyDrive/Optum/images/train/8881...,8881c07203fc,NaN,none 1 0 0 1 1,b4dcf0375a27,0
1,/content/drive/MyDrive/Optum/images/train/0893...,08932dca1447,NaN,none 1 0 0 1 1,f109811743c5,0
2,/content/drive/MyDrive/Optum/images/train/8866...,8866f708d338,"[{'x': 1559.4013, 'y': 50.88, 'width': 1280.48...",opacity 1 1559.4013 50.88 2839.88143 1772.3200...,6bc01f9a44a5,1
3,/content/drive/MyDrive/Optum/images/train/8951...,8951eac18958,"[{'x': 583.55566, 'y': 1586.66667, 'width': 14...",opacity 1 583.55566 1586.66667 2002.22233 3165...,a0927e51897c,1
4,/content/drive/MyDrive/Optum/images/train/8925...,8925b615fc62,"[{'x': 1592.45982, 'y': 1223.98355, 'width': 4...",opacity 1 1592.45982 1223.98355 2029.73948 178...,cbf3b338fae0,2


In [ ]:
#Checking for null values in each column
complete.isnull().sum()


Location               0
id                     0
boxes               2040
label                  0
StudyInstanceUID       0
result                 0
dtype: int64

In [ ]:
# Filling the NaN values with 0
complete['boxes'].fillna(0,inplace=True)

In [ ]:
complete.head()

,Location,id,boxes,label,StudyInstanceUID,result
0,/content/drive/MyDrive/Optum/images/train/8881...,8881c07203fc,0,none 1 0 0 1 1,b4dcf0375a27,0
1,/content/drive/MyDrive/Optum/images/train/0893...,08932dca1447,0,none 1 0 0 1 1,f109811743c5,0
2,/content/drive/MyDrive/Optum/images/train/8866...,8866f708d338,"[{'x': 1559.4013, 'y': 50.88, 'width': 1280.48...",opacity 1 1559.4013 50.88 2839.88143 1772.3200...,6bc01f9a44a5,1
3,/content/drive/MyDrive/Optum/images/train/8951...,8951eac18958,"[{'x': 583.55566, 'y': 1586.66667, 'width': 14...",opacity 1 583.55566 1586.66667 2002.22233 3165...,a0927e51897c,1
4,/content/drive/MyDrive/Optum/images/train/8925...,8925b615fc62,"[{'x': 1592.45982, 'y': 1223.98355, 'width': 4...",opacity 1 1592.45982 1223.98355 2029.73948 178...,cbf3b338fae0,2


In [ ]:
# Count of different class labels
complete.result.value_counts()

1    3007
0    1736
2    1108
3     483
Name: result, dtype: int64

In [ ]:
f=["boxes","label"]
features=complete[f]

In [ ]:
t= features.loc[3,"boxes"]
print(t)

[{'x': 583.55566, 'y': 1586.66667, 'width': 1418.66667, 'height': 1578.66667}, {'x': 2514.22233, 'y': 1282.66666, 'width': 1274.66667, 'height': 1920}]


In [ ]:
import ast
res = ast.literal_eval(t)#ast.literal_eval(source) with source as a string or node containing an expression to evaluate the expression.
print(res)

[{'x': 583.55566, 'y': 1586.66667, 'width': 1418.66667, 'height': 1578.66667}, {'x': 2514.22233, 'y': 1282.66666, 'width': 1274.66667, 'height': 1920}]


In [ ]:
import torch
from IPython.display import Image, clear_output
import PIL
from PIL import Image

In [ ]:
#create lists of x,y,width and height

X=[]
Y=[]
X_max=[]
Y_max=[]

for i in range(len(complete)):
  t=complete.loc[i,"boxes"]
  sample=complete.loc[i,"label"]

  if t==0:
    X.append(0.0)
    Y.append(0.0)
    X_max.append(0.0)
    Y_max.append(0.0)
  else:
    lis_dic=ast.literal_eval(t)
    image=PIL.Image.open(complete.loc[i,"Location"])
    width,height=image.size
    scale_x=(640/width)
    scale_y=(640/height)

    sample=sample.split("opacity 1 ")
    temp=sample[1]
    temp=temp.split(" ")

    for i in range(1):
      x=int(np.round(float(temp[0])*scale_x,4))
      y=int(np.round(float(temp[1])*scale_y,4))
      w=int(np.round(float(temp[2])*scale_x,4))
      h=int(np.round(float(temp[3])*scale_y,4))
      w = w - x # xmax - xmin
      h = h - y # ymax - ymin
      xc = x + int(np.round(w/2)) # xmin + width/2
      yc = y + int(np.round(h/2)) # ymin + height/2
    X.append(round(xc/640,6))
    Y.append(round(yc/640,6))
    X_max.append(round(w/640,6))
    Y_max.append(round(h/640,6))

In [ ]:
#create columns of x,y,x_max,y_max

X=pd.DataFrame(X);
X["y"]=pd.DataFrame(Y);
X["x_max"]=pd.DataFrame(X_max)
X["y_max"]=pd.DataFrame(Y_max)

In [ ]:
# Renaming the first column
X=X.rename({0: 'x'}, axis=1)
X.head()

,x,y,x_max,y_max
0,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000
2,0.718750,0.356250,0.418750,0.676562
3,0.303125,0.682813,0.334375,0.454688
4,0.745313,0.618750,0.181250,0.231250


In [ ]:
Y=complete.result

In [ ]:
#Printing shape
print(X.shape)
print(Y.shape)

(6334, 4)
(6334,)


# SVM

In [ ]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
import pandas as pd

In [ ]:
#Use train_test_split() to get training and test sets
#Determine the randomness of your splits with the random_state parameter
#Obtain stratified splits with the stratify parameter
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, stratify=Y, random_state=42)

In [ ]:
from sklearn.svm import SVC
from sklearn import metrics

In [ ]:
svm=SVC(kernel="rbf")
#fit the model on training set
svm.fit(X_train,Y_train)
prediction=svm.predict(X_test)
#Printing the accuracy score
print(metrics.accuracy_score(prediction,Y_test)*100)

73.80326144134666


In [ ]:
from sklearn.externals import joblib

/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
#Save the model to drive
joblib.dump(svm, "/content/drive/MyDrive/Optum/model.pkl")

['model.pkl']

# Decision Tree Classifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn import metrics

In [ ]:
mod_dt = DecisionTreeClassifier(max_depth = 3, random_state = 1)
#fit the model on training set
mod_dt.fit(X_train,Y_train)
prediction=mod_dt.predict(X_test)
#Printing the accuracy score
print(metrics.accuracy_score(prediction,Y_test)*100)

73.17201472908995


# K- Nearest Neighbour Classifier

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

In [ ]:
knn=KNeighborsClassifier(n_neighbors=15)
#fit the model on training set
knn.fit(X_train,Y_train)
prediction=knn.predict(X_test)
#Printing the accuracy score
print(metrics.accuracy_score(prediction,Y_test)*100)

73.48763808521831


# Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

In [ ]:
rfm=RandomForestClassifier(n_estimators=12)
#fit the model on training set
rfm.fit(X_train,Y_train)
prediction=rfm.predict(X_test)
#Printing the accuracy score
print(metrics.accuracy_score(prediction,Y_test)*100)

72.69857969489742


# CNN

In [ ]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
import pandas as pd
#
# Import Keras modules
#
from keras import models
from keras import layers
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,Flatten,MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau

In [ ]:
#Building the model
network = models.Sequential()
network.add(layers.Dense(512, activation='relu', input_shape=(4,)))
network.add(layers.Dense(4, activation='softmax'))

In [ ]:
#model compilation
network.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

In [ ]:
#Use train_test_split() to get training and test sets
#Determine the randomness of your splits with the random_state parameter
#Obtain stratified splits with the stratify parameter

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, stratify=Y, random_state=42)

In [ ]:
#Using the method to_categorical(), a numpy array (or) a vector which has integers that represent different categories, can be converted into a numpy array (or) a matrix which has binary values and has columns equal to the number of categories in the data.
train_labels = to_categorical(Y_train)
test_labels = to_categorical(Y_test)

In [ ]:
#Fit the model instances with .fit() using the training set.
network.fit(X_train,train_labels, epochs=100, batch_size=40)


Epoch 1/100
111/111 [==============================] - 15s 2ms/step - loss: 1.1107 - accuracy: 0.5327
Epoch 2/100
111/111 [==============================] - 0s 1ms/step - loss: 0.7577 - accuracy: 0.7231
Epoch 3/100
111/111 [==============================] - 0s 1ms/step - loss: 0.6940 - accuracy: 0.7299
Epoch 4/100
111/111 [==============================] - 0s 1ms/step - loss: 0.6934 - accuracy: 0.7334
Epoch 5/100
111/111 [==============================] - 0s 2ms/step - loss: 0.7154 - accuracy: 0.7319
Epoch 6/100
111/111 [==============================] - 0s 1ms/step - loss: 0.7039 - accuracy: 0.7349
Epoch 7/100
111/111 [==============================] - 0s 1ms/step - loss: 0.6954 - accuracy: 0.7337
Epoch 8/100
111/111 [==============================] - 0s 1ms/step - loss: 0.6891 - accuracy: 0.7414
Epoch 9/100
111/111 [==============================] - 0s 1ms/step - loss: 0.6765 - accuracy: 0.7475
Epoch 10/100
111/111 [==============================] - 0s 2ms/step - loss: 0.6788 - accur

In [ ]:
# Get the accuracy of test data set
test_loss, test_acc = network.evaluate(X_test, test_labels)

# Print the test accuracy
print('Test Accuracy: ', test_acc, '\nTest Loss: ', test_loss)

60/60 [==============================] - 1s 1ms/step - loss: 0.6954 - accuracy: 0.7386
Test Accuracy:  0.7385586500167847 
Test Loss:  0.695377767086029


# Overall Accuracy

In [ ]:
# saving Model with joblib

model = joblib.load("model.pkl")

In [ ]:
len(data)

6334

In [ ]:
path = '/content/drive/MyDrive/Optum/images/test'
 
d=Path(path)
# This would print all the files and directories
images=[]
for files in d.glob('*.png'): #glob module is used to retrieve files/pathnames matching a specified pattern.
  images.append(files)

In [ ]:
img_loc=pd.DataFrame()
img_loc['Location']=images
img_loc['id']=img_loc.apply(lambda row:str(row.Location)[-16:-4],axis=1)
img_loc.head()

,Location,id
0,/content/drive/MyDrive/Optum/images/test/ff766...,ff7663694713
1,/content/drive/MyDrive/Optum/images/test/ff231...,ff23167d20b4
2,/content/drive/MyDrive/Optum/images/test/ff765...,ff7659762b75
3,/content/drive/MyDrive/Optum/images/test/ffab0...,ffab0f8f27f0
4,/content/drive/MyDrive/Optum/images/test/ffcc6...,ffcc6edd9445


In [ ]:
test_set=pd.merge(left=img_loc, right=data, left_on='id', right_on='id_x')
test_set.head(10)

,Location,id,id_x,boxes,label,StudyInstanceUID,id_y,result
0,/content/drive/MyDrive/Optum/images/test/ff766...,ff7663694713,ff7663694713,"[{'x': 413.51042, 'y': 1883.22786, 'width': 35...",opacity 1 413.51042 1883.22786 768.50520000000...,386aa41785fa,386aa41785fa,1
1,/content/drive/MyDrive/Optum/images/test/ff231...,ff23167d20b4,ff23167d20b4,"[{'x': 944.39193, 'y': 1126.749, 'width': 526....",opacity 1 944.39193 1126.749 1470.52737 1938.0...,96d1f9ad6e80,96d1f9ad6e80,1
2,/content/drive/MyDrive/Optum/images/test/ff765...,ff7659762b75,ff7659762b75,NaN,none 1 0 0 1 1,4aa11ef1fd38,4aa11ef1fd38,0
3,/content/drive/MyDrive/Optum/images/test/ffab0...,ffab0f8f27f0,ffab0f8f27f0,"[{'x': 226.1586, 'y': 853.30535, 'width': 571....",opacity 1 226.1586 853.30535 797.83728 1792.49...,a5af28176585,a5af28176585,1
4,/content/drive/MyDrive/Optum/images/test/ffcc6...,ffcc6edd9445,ffcc6edd9445,NaN,none 1 0 0 1 1,7e6c68462e06,7e6c68462e06,0
5,/content/drive/MyDrive/Optum/images/test/ffd9b...,ffd9b6cf2961,ffd9b6cf2961,"[{'x': 2197.38566, 'y': 841.07361, 'width': 31...",opacity 1 2197.38566 841.07361 2513.80265 1292...,7eed9af03814,7eed9af03814,1
6,/content/drive/MyDrive/Optum/images/test/ffa9f...,ffa9fef3c7bf,ffa9fef3c7bf,NaN,none 1 0 0 1 1,5133bbb7b75c,5133bbb7b75c,0
7,/content/drive/MyDrive/Optum/images/test/ffcc1...,ffcc16bbf428,ffcc16bbf428,"[{'x': 2529.03977, 'y': 347.67053, 'width': 58...",opacity 1 2529.03977 347.67053 3113.65526 1484...,5d786634f354,5d786634f354,1
8,/content/drive/MyDrive/Optum/images/test/ffbea...,ffbeafe30b77,ffbeafe30b77,"[{'x': 1472.09879, 'y': 1182.64415, 'width': 3...",opacity 1 1472.09879 1182.64415 1773.17398 146...,f75774536369,f75774536369,1
9,/content/drive/MyDrive/Optum/images/test/ff012...,ff01229b525c,ff01229b525c,NaN,none 1 0 0 1 1,7f88c3e4abca,7f88c3e4abca,0


In [ ]:
test_set.drop(columns=['id_x','id_y'],inplace=True)
test_set.head()

,Location,id,boxes,label,StudyInstanceUID,result
0,/content/drive/MyDrive/Optum/images/test/ff766...,ff7663694713,"[{'x': 413.51042, 'y': 1883.22786, 'width': 35...",opacity 1 413.51042 1883.22786 768.50520000000...,386aa41785fa,1
1,/content/drive/MyDrive/Optum/images/test/ff231...,ff23167d20b4,"[{'x': 944.39193, 'y': 1126.749, 'width': 526....",opacity 1 944.39193 1126.749 1470.52737 1938.0...,96d1f9ad6e80,1
2,/content/drive/MyDrive/Optum/images/test/ff765...,ff7659762b75,NaN,none 1 0 0 1 1,4aa11ef1fd38,0
3,/content/drive/MyDrive/Optum/images/test/ffab0...,ffab0f8f27f0,"[{'x': 226.1586, 'y': 853.30535, 'width': 571....",opacity 1 226.1586 853.30535 797.83728 1792.49...,a5af28176585,1
4,/content/drive/MyDrive/Optum/images/test/ffcc6...,ffcc6edd9445,NaN,none 1 0 0 1 1,7e6c68462e06,0


In [ ]:
test_set['boxes'].fillna(0,inplace=True)

In [ ]:
y_test=test_set.result

In [ ]:
import cv2
import csv 
import json
import random

In [ ]:
#class 0-->Pneumonia 
#class 1-->Typical Appearance 
#class 2-->Indeterminate Appearance 
#class 3-->Atypical Appearance

def predictImage(filename):
  img_file=filename
  f_name=filename[-16:]

   #tf_name[:-4] -> name of the label text file

  fname=f_name[:-4]
  n = random.randint(0,22)
  fname=fname+str(n)

  #fname -> name of the folder where the result of test image is stored

  result=['Negative for Pneumonia','Typical Appearance','Indeterminate Appearance','Atypical Appearance']

  #predict the bounding box values
  #call detect.py , pass the weights,image path,result storage path

  output=!python '/content/drive/MyDrive/Optum/yolov5/detect.py' --weights '/content/drive/MyDrive/Optum/yolov5/runs/train/exp8/weights/best.pt' --source {img_file}  --img 640 --conf 0.281 --project 'runs/detect' --name {fname} --save-txt
  print(output)

   #path of the text file
  
  txt_file="/content/runs/detect/"+ fname + "/labels/"+f_name[:-4]+".txt"
  
  if os.path.isfile(txt_file):

    #read labels from the text file

    f = open(txt_file, "r")
    res=f.readline()
    res=res.rstrip('\n')
    res=res[2:]
    X_pred=res.split(" ")

    #store labels in a list

    for i in range(4):
      X_pred[i]=float(X_pred[i])

     #convert it into 2-D np array
    
    X_pred=np.array(X_pred)
    X_pred = np.reshape(X_pred, (1, 4))
    prediction=svm.predict(X_pred)
    
  else:
    #labels for class none

    X_pred=[0.0,0.0,0.0,0.0]
    X_pred=np.array(X_pred)

    #convert it into 2-D np array

    X_pred = np.reshape(X_pred, (1, 4))

    
    #predict result using the svm model

    prediction=svm.predict(X_pred)

  
  #print the result

  print(prediction)

  #return the class name of the result

  return prediction[0]

In [ ]:
y_pred=[]
for i in range (len(test_set)):
  y_pred.append(predictImage(str(test_set.loc[i,"Location"])))


["\x1b\x1bdetect: \x1bweights=['/content/drive/MyDrive/Optum/yolov5/runs/train/exp8/weights/best.pt'], source=/content/drive/MyDrive/Optum/images/test/ff7663694713.png, imgsz=640, conf_thres=0.281, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=ff76636947135, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False", '\x1b\x1brequirements:\x1b /content/requirements.txt not found, check failed.', 'YOLOv5 🚀 v5.0-319-g63a1971 torch 1.9.0+cu102 CPU', '', 'Fusing layers... ', 'Model Summary: 224 layers, 7056607 parameters, 0 gradients', '/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Tri

detect: weights=['/content/drive/MyDrive/Optum/yolov5/runs/train/exp8/weights/best.pt'], source=/content/drive/MyDrive/Optum/images/test/f658eafd6b22.png, imgsz=640, conf_thres=0.281, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=f658eafd6b2216, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False
requirements: /content/requirements.txt not found, check failed.
YOLOv5 🚀 v5.0-319-g63a1971 torch 1.9.0+cu102 CPU

Fusing layers... 
Model Summary: 224 layers, 7056607 parameters, 0 gradients
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/Ten

In [ ]:
print(metrics.accuracy_score(y_pred,y_test)*100)

52.76461295418642
